### Load and Analyze the Image

In [1]:
import ollama

In [15]:
image_path = r'pics\Receipt.jpg'  # Replace with your image path

# Use Ollama to analyze the image with Llama 3.2-Vision
response = ollama.chat(
    model="llama3.2-vision",
    messages=[{
      "role": "user",
      "content": "get all the data from the image",
      "images": [image_path]
    }],
)


In [16]:
#response['message']['content']
# Extract the model's response about the image
cleaned_text = response['message']['content'].strip()
print(f"Model Response: {cleaned_text}")

Model Response: Here is a detailed breakdown of the information presented in the image:

**Invoice Information**

* **Company Name:** East Repair Inc.
* **Address:** 1912 Harvest Lane, New York NY 12210
* **Bill To:**
	+ Name: John Smith
	+ Address: 3787 Pineview Drive, Cambridge MA 02240
* **Ship To:**
	+ Same as Bill To

**Invoice Details**

* **Invoice Number:** Not provided
* **Date:** February 11th, 2019 (US-001)
* **Due Date:** February 26th, 2019
* **Payment Terms:** Payment is due within 15 days

**Items Purchased**

* **Front and Rear Brake Cables:** $100.00
* **New Set of Pedal Arms:** $30.00
* **Labor (3 hours):** $15.00

**Subtotal:** $145.00
**Sales Tax (6.25%):** $9.06
**Total:** $154.06

**Payment Instructions**

* **Method:** Bank Transfer (ABC 0560120214)
* **Payee:** East Repair Inc.

**Terms and Conditions**

* Payment is due within 15 days
* Please make checks payable to: East Repair Inc.

This breakdown provides a clear and organized summary of the invoice informat

### Structured JSON Output

In [17]:
from langchain_ollama import ChatOllama
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

In [18]:
# Define the template
def get_response(response):
        #input_variables=["response"],
    template=""" Extract the following receipt details from the provided text response and return them as a structured JSON object.

    Fields to extract:
    - Company
    - Bill To (Name, Address)
    - Ship To (Name, Address)
    - Receipt Number
    - Date
    - P.O. Number
    - Items (Description, Quantity, Unit Price, Total)
    - Subtotal
    - Sales Tax
    - Total
    - Payment Instructions (Bank Transfer, Paypal email)
    - Terms & Conditions

    Input text:
    {response}

    Return only JSON, no extra text or explanations.
    """
    
#
    prompt = ChatPromptTemplate.from_template(template)

    # llm
    llm = ChatOllama(model="llama3",  temperature=0)

    chain = ( prompt
    | llm
    | StrOutputParser()
    )

    return chain.invoke({
    "response": response,
    })


In [19]:
result = get_response(cleaned_text)

In [20]:
result

'Here is the extracted receipt details in a structured JSON object:\n\n```\n{\n  "Company": "East Repair Inc.",\n  "Bill To": {\n    "Name": "John Smith",\n    "Address": "3787 Pineview Drive, Cambridge MA 02240"\n  },\n  "Ship To": {\n    "Name": "John Smith",\n    "Address": "3787 Pineview Drive, Cambridge MA 02240"\n  },\n  "Receipt Number": null,\n  "Date": "February 11th, 2019",\n  "P. O. Number": null,\n  "Items": [\n    {\n      "Description": "Front and Rear Brake Cables",\n      "Quantity": null,\n      "Unit Price": 100.0,\n      "Total": 100.0\n    },\n    {\n      "Description": "New Set of Pedal Arms",\n      "Quantity": null,\n      "Unit Price": 30.0,\n      "Total": 30.0\n    },\n    {\n      "Description": "Labor (3 hours)",\n      "Quantity": null,\n      "Unit Price": 15.0,\n      "Total": 15.0\n    }\n  ],\n  "Subtotal": 145.0,\n  "Sales Tax": 9.06,\n  "Total": 154.06,\n  "Payment Instructions": {\n    "Method": "Bank Transfer",\n    "Payee": "East Repair Inc.",\n  

###  Extract the JSON with Regex

In [21]:
import re
json_match = re.search(r"```\n(.*?)\n```", result, re.DOTALL)

if json_match:
    receipt_data = json_match.group(1)
    try:
        parsed_data = json.loads(receipt_data)
        json_data = json.dumps(parsed_data, indent=2)
        print(json.dumps(parsed_data, indent=2))
    except json.JSONDecodeError:
        print("Failed to parse JSON.")
else:
    print("No JSON found.")

{
  "Company": "East Repair Inc.",
  "Bill To": {
    "Name": "John Smith",
    "Address": "3787 Pineview Drive, Cambridge MA 02240"
  },
  "Ship To": {
    "Name": "John Smith",
    "Address": "3787 Pineview Drive, Cambridge MA 02240"
  },
  "Receipt Number": null,
  "Date": "February 11th, 2019",
  "P. O. Number": null,
  "Items": [
    {
      "Description": "Front and Rear Brake Cables",
      "Quantity": null,
      "Unit Price": 100.0,
      "Total": 100.0
    },
    {
      "Description": "New Set of Pedal Arms",
      "Quantity": null,
      "Unit Price": 30.0,
      "Total": 30.0
    },
    {
      "Description": "Labor (3 hours)",
      "Quantity": null,
      "Unit Price": 15.0,
      "Total": 15.0
    }
  ],
  "Subtotal": 145.0,
  "Sales Tax": 9.06,
  "Total": 154.06,
  "Payment Instructions": {
    "Method": "Bank Transfer",
    "Payee": "East Repair Inc.",
    "Account Number": "ABC 0560120214"
  },
  "Terms & Conditions": [
    "Payment is due within 15 days",
    "Pleas

### Convert to json to a Pandas DataFrame

In [22]:
import pandas as pd
import json
#
receipt_dict = json.loads(json_data)
receipt_dict

{'Company': 'East Repair Inc.',
 'Bill To': {'Name': 'John Smith',
  'Address': '3787 Pineview Drive, Cambridge MA 02240'},
 'Ship To': {'Name': 'John Smith',
  'Address': '3787 Pineview Drive, Cambridge MA 02240'},
 'Receipt Number': None,
 'Date': 'February 11th, 2019',
 'P. O. Number': None,
 'Items': [{'Description': 'Front and Rear Brake Cables',
   'Quantity': None,
   'Unit Price': 100.0,
   'Total': 100.0},
  {'Description': 'New Set of Pedal Arms',
   'Quantity': None,
   'Unit Price': 30.0,
   'Total': 30.0},
  {'Description': 'Labor (3 hours)',
   'Quantity': None,
   'Unit Price': 15.0,
   'Total': 15.0}],
 'Subtotal': 145.0,
 'Sales Tax': 9.06,
 'Total': 154.06,
 'Payment Instructions': {'Method': 'Bank Transfer',
  'Payee': 'East Repair Inc.',
  'Account Number': 'ABC 0560120214'},
 'Terms & Conditions': ['Payment is due within 15 days',
  'Please make checks payable to: East Repair Inc.']}

In [23]:
items_df = pd.DataFrame(receipt_dict['Items'])
items_df

,Description,Quantity,Unit Price,Total
0,Front and Rear Brake Cables,None,100.0,100.0
1,New Set of Pedal Arms,None,30.0,30.0
2,Labor (3 hours),None,15.0,15.0


In [24]:
# Optionally, add general receipt details as columns
receipt_details = {
    "Company": receipt_dict['Company'],
    "Bill To": f"{receipt_dict['Bill To']['Name']}, {receipt_dict['Bill To']['Address']}",
    "Receipt Number": receipt_dict['Receipt Number'],
    "Date": receipt_dict['Date'],
    "Subtotal": receipt_dict['Subtotal'],
    "Tax": receipt_dict['Sales Tax'],
    "Total": receipt_dict['Total'],
    "Payment Method": receipt_dict['Payment Instructions']['Method']
}

In [25]:
# Add receipt details as columns to each item row
for key, value in receipt_details.items():
    items_df[key] = value
#
items_df

,Description,Quantity,Unit Price,Total,Company,Bill To,Receipt Number,Date,Subtotal,Tax,Payment Method
0,Front and Rear Brake Cables,None,100.0,154.06,East Repair Inc.,"John Smith, 3787 Pineview Drive, Cambridge MA ...",None,"February 11th, 2019",145.0,9.06,Bank Transfer
1,New Set of Pedal Arms,None,30.0,154.06,East Repair Inc.,"John Smith, 3787 Pineview Drive, Cambridge MA ...",None,"February 11th, 2019",145.0,9.06,Bank Transfer
2,Labor (3 hours),None,15.0,154.06,East Repair Inc.,"John Smith, 3787 Pineview Drive, Cambridge MA ...",None,"February 11th, 2019",145.0,9.06,Bank Transfer


In [13]:
#items_df['Line Total'] = items_df['Quantity'] * items_df['Unit Price']

In [26]:
#items_df